--Download library--

In [1]:
!pip install corextopic

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


--import dependencies--

In [2]:
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_lg

from pprint import pprint

## Setup NLP Pipeline and Data Cleaning 

In [3]:
##define nlp and start cleaning data
nlp= spacy.load("en")

# List of stop words to equalize data
stop_list = ["Depression","depression",  "anxiety", "Anxiety"]

# Updates spaCy's default stop words list with my additional words. 
nlp.Defaults.stop_words.update(stop_list)

# Iterates over the words in the stop words list and resets the "is_stop" flag.
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True


In [4]:
#clean data

def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
    
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

# The add_pipe function appends our functions to the default pipeline.
nlp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
nlp.add_pipe(remove_stopwords, name="stopwords", last=True)

## import depression data

In [5]:
doc_dep = pd.read_csv(r'Datasets/2020_March_r_Depression.csv')
doc_dep = doc_dep[doc_dep.Body != '[removed]']
doc_dep = doc_dep.sample(n=7154) 

In [6]:
#create a list of documents (list of lists) 
text_doc = doc_dep['Body'].tolist()

In [7]:
doc_list = []
# Iterates through each article in the corpus.
for doc in text_doc:
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(str(doc))
    doc_list.append(pr)

In [9]:
#However doc_list gives us each post as a list, with individual words being elements 
from nltk.tokenize.treebank import TreebankWordDetokenizer
empt = [];
for doc in doc_list:
    a1 = TreebankWordDetokenizer().detokenize(doc)
    empt.append(a1)
    
df_doc_dep = pd.DataFrame(empt,columns = ['Body'])
#df_doc_dep is a dataframe that has cleaned posts from Depression subreddit.
#all the 'removed' posts are gone and all the stopwords in the individual posts are gone! 

#both doc_list and df_doc are important (at least I think so :D)


df_doc_dep['label'] = 0

In [10]:
df_doc_dep

,Body,label
0,gen immigrant sister send parent 14 22 parent ...,0
1,feel happy think friend grouo want,0
2,anger stress sadness guilt build year release ...,0
3,okay girl kind weird history know super deep e...,0
4,hello like know drug good fast painless death,0
...,...,...
7149,know long term thing ok right coronavirus blow...,0
7150,work incredibly hard career 16 successful rega...,0
7151,long remember feel feel happy hear remember ha...,0
7152,actually great weekend good family time great ...,0


--read in data--

## anxietydataset

In [11]:

doc_anx = pd.read_csv(r'2020_March_r_Anxiety.csv')
doc_anx = doc_anx[doc_anx.Body != '[removed]']

text_doc_anx = doc_anx['Body'].tolist()

In [12]:
doc_list_anx = []
# Iterates through each article in the corpus.
for doc in text_doc_anx:
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(str(doc))
    doc_list_anx.append(pr)

## from tokens create document list again

In [13]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
empt = [];
for doc in doc_list_anx:
    a1 = TreebankWordDetokenizer().detokenize(doc)
    empt.append(a1)
    
df_doc_anx = pd.DataFrame(empt,columns = ['Body'])
#df_doc_anx is a dataframe that has cleaned posts from Anxiety subreddit
#all the 'removed' posts are gone and all the stopwords in the individual posts are gone! 

#both doc_list and df_doc are important (at least I think so :D)

df_doc_anx['label']=1
df_doc_anx

,Body,label
0,thing anxious focus mean study like maybe lazy...,1
1,know midnight know homework know school know X...,1
2,med euphoric feeling consider narcotic,1
3,heart feel sense yesterday continually worried...,1
4,fear swallow small bone eat chicken sure bite ...,1
...,...,...
7149,want sell ps4 use anymore want money hard pers...,1
7150,yesterday(this morning sleep midnight 12 wake ...,1
7151,bath new house cause flood basement live boyfr...,1
7152,mean find self cross leg,1


## Merge Depression and Anxiety subreddits 

In [14]:
df_col_merged =pd.concat([df_doc_dep, df_doc_anx], axis=0).reset_index(drop=True)
df_col_merged

,Body,label
0,gen immigrant sister send parent 14 22 parent ...,0
1,feel happy think friend grouo want,0
2,anger stress sadness guilt build year release ...,0
3,okay girl kind weird history know super deep e...,0
4,hello like know drug good fast painless death,0
...,...,...
14303,want sell ps4 use anymore want money hard pers...,1
14304,yesterday(this morning sleep midnight 12 wake ...,1
14305,bath new house cause flood basement live boyfr...,1
14306,mean find self cross leg,1


## Obtain TF-IDF score for combined dataframes

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_df=.2,
    min_df=8,
    max_features=None,
    ngram_range=(1, 1),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)

vectorizer = vectorizer.fit(df_col_merged.Body)
tfidf = vectorizer.transform(df_col_merged.Body)
vocab = vectorizer.get_feature_names()
print(len(vocab))

5118


## Machine Learning

In [31]:
X = tfidf;
y = df_col_merged.label.tolist()

import numpy as np
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X_train, y_train)


## clf.score provides the accuracy of machine learning model

In [32]:
clf.score(X_test,y_test)

0.7959468902865129

## Topic Modelling

In [ ]:
#read in data
doc = pd.read_csv(r'Datasets/2020_March_r_Depression.csv')
doc = doc[doc.Body != '[removed]']

In [ ]:
#create a list of documents (list of lists) 
text_doc = doc['Body'].tolist()

In [ ]:
nlp= spacy.load("en")

# List of stop words to equalize data
stop_list = ["Depression","depression", "coronavirus", "quarantine", "coronavirus", "Coronavirus", "lockdown", "anxiety", "Anxiety", "Quarantine", "Lockdown", "Agoraphobia", "Agoraphobic", "agoraphobic", "agoraphobia"]

# Updates spaCy's default stop words list with my additional words. 
nlp.Defaults.stop_words.update(stop_list)

# Iterates over the words in the stop words list and resets the "is_stop" flag.
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

In [ ]:
def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
    
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

# The add_pipe function appends our functions to the default pipeline.
nlp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
nlp.add_pipe(remove_stopwords, name="stopwords", last=True)



In [ ]:
doc_list = []
# Iterates through each article in the corpus.
for doc in text_doc:
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(str(doc))
    doc_list.append(pr)

In [ ]:
#however doc_list gives us each post as a list, with individual words being elements 
from nltk.tokenize.treebank import TreebankWordDetokenizer
empt = [];
for doc in doc_list:
    a1 = TreebankWordDetokenizer().detokenize(doc)
    empt.append(a1)
    
df_doc = pd.DataFrame(empt,columns = ['Body'])
#df_doc is a dataframe that has cleaned posts. 
#all the 'removed' posts are gone and all the stopwords in the individual posts are gone! 

#both doc_list and df_doc are important (at least I think so :D)


df_doc

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_df=.5,
    min_df=10,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)
vectorizer = vectorizer.fit(df_doc.Body)
tfidf = vectorizer.transform(df_doc.Body)
vocab = vectorizer.get_feature_names()
print(len(vocab))

In [ ]:
#import corextopic

from corextopic import corextopic as ct
anchors = []
model = ct.Corex(n_hidden=6, seed=42)
model = model.fit(
    tfidf,
    words=vocab
)

In [ ]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=15)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

In [ ]:
# Anchors designed to nudge the model towards measuring specific genres
anchors = [
    ["family"],
    ["die"],
    [""],
    [""],
    [""],
    [""],

]
anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=8, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=3 # Tell the model how much it should rely on the anchors
)

In [ ]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

In [ ]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    columns=["topic_{}".format(i+1) for i in range(6)]
).astype(float)
topic_df.iloc[0:10]

In [ ]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    columns=["topic_{}".format(i+1) for i in range(6)]
).astype(float)
topic_df

## LDA Unsupervised

In [ ]:
# Creates, which is a mapping of word IDs to words.
words = corpora.Dictionary(doc_list)

# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in doc_list]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=6, 
                                           random_state=2,
                                           update_every=1,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
#print the keyword in the 10 topics
pprint(lda_model.print_topics(num_words=100))

In [ ]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=doc_list):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=doc_list)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(100)

In [ ]:
df_dominant_topic.head(25)
#df_dominant_topic.to_csv(r'depression_2020_datest.csv')

In [ ]:
from collections import Counter
Counter(df_dominant_topic.Dominant_Topic)

In [ ]:
newest_doc = newest_doc[newest_doc.Body != '[removed]'].dropna()

In [ ]:
newest_doc.Body

In [ ]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    columns=["topic_{}".format(i+1) for i in range(6)]
).astype(float)

df = pd.concat([df, topic_df], axis=1)

In [ ]:
topic_df